In [15]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
import math
import psycopg2 as psy
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import pandas.io.sql as psql
%matplotlib inline


In [5]:
c1_targets = pd.read_csv('/Users/cguy/kelstuff/ga_summer_2016/DSI-projects/Projects_Weekly/Capstone_Project/2_EDA/cohort1_targets_FV.csv')
c1_features = pd.read_csv('/Users/cguy/kelstuff/ga_summer_2016/DSI-projects/Projects_Weekly/Capstone_Project/2_EDA/cohort1_features_FV.csv')


In [13]:
c1_targets = c1_targets.drop('Unnamed: 0', axis=1)
c1_targets.head()

,PubFoodAs,PubHousAs,OthrFinAs,HighEdCert,CurrFTE,CurrPTE,PubFinAs,CurrenRoll,PrescripIn,Homeless,SubAbuse,Incarc,Medicaid,CnctAdult,EmplySklls,OthrHlthIn,MedicalIn,MentlHlthIn
0,Yes,No,No,None of the above,No,No,No,No,Not Applicable,Yes,No,No,Yes,Yes,No,No,Not Applicable,Not Applicable
1,Yes,No,No,High school diploma/GED,No,No,No,Yes,Not Applicable,No,Yes,Yes,Yes,Yes,No,No,Not Applicable,Not Applicable
2,Yes,No,No,High school diploma/GED,No,"Yes, employed part time",No,No,Not Applicable,Yes,No,No,Yes,Yes,No,No,Not Applicable,Not Applicable
3,Yes,No,No,High school diploma/GED,"Yes, employed full time",No,No,No,Not Applicable,Yes,No,No,Yes,Yes,No,Yes,Don't Know,Not Applicable
4,Not Applicable (in Foster Care),Not Applicable (in Foster Care),No,None of the above,No,No,Not Applicable (in Foster Care),Yes,Not Applicable,No,No,No,Yes,Yes,No,No,Not Applicable,Not Applicable


In [14]:
c1_features = c1_features.drop('Unnamed: 0', axis=1)
c1_features.head()

,ILNAsv,AcSuppSv,PSEdSuppSv,CareerSv,EmplyTrSv,BudgetSv,HousEdSv,HlthEdSv,FamSuppSv,MentorSv,...,EducFinaSv,OthrFinaSv,St,DelinqntSv,Sex,Race,TribeSv,EdLevlSv,SpecEdSv,RaceEthn
0,0,0,0,0,0,0,1,0,0,0,...,0,1,AK,0,2,1,0,11,0,1
1,1,0,0,0,0,0,0,0,0,0,...,1,1,AK,0,2,1,0,11,0,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,AK,0,1,1,0,11,0,1
3,1,0,0,0,0,0,0,0,0,0,...,0,1,AK,0,2,3,1,11,1,3
4,0,0,0,0,0,0,0,0,0,0,...,0,1,AK,0,1,1,0,9,1,7


# Preprocessing: Optimal Scaling of target variables in order to conduct PCA

##### The code below is from Alex Lubbock and can be found in the following github repository:
https://github.com/alubbock/pyopscale/blob/master/pyopscale/opscale.py


In [59]:
# Borrowed from alubbock:

import rpy2.robjects as ro
import os


class OpScale:
    def __init__(self):
        self.r = ro.r
        self.r.source('/Users/cguy/kelstuff/ga_summer_2016/DSI-projects/Projects_Weekly/Capstone_Project/opscale.R')

    def opscale(self, qual, quant=None, level=1, process=1, na_impute=False,
                na_assign=False, rescale=True):
        """
        :param qual: A vector of data values, assumed nominal or
                   ordinal level measurement
        :param quant: A vector of quantitative values, probably
                   obtained as model estimates
        :param level: Measurement level of qualitative vector, with
                   1 = nominal and 2 = ordinal
        :param process: Measurement process of qualitative vector, with
                   1 = discrete and 2 = continuous
        :param na_impute: FALSE (default) = leave missing values in qualitative
                   vector as missing in optimally scaled vector,
                   TRUE = assign quantitative vector values to
                   optimally scaled vector for missing entries in
                   qualitative vector
        :param na_assign: TRUE (default) = if quantitative value is missing,
                   assign mean of optimally scaled values for the corresponding
                   qualitative value to the optimally scaled vector,
                   FALSE = if quantitative value is missing, leave optimally
                   scaled value missing, even if qualitative value is present
        :param rescale:TRUE (default) = rescale optimally scaled values to the
                   same mean and standard deviation as the values in the
                   original qualitative vector.  FALSE = do not rescale
                   optimally scaled vector
        :return: Optimally scaled version of the qualitative vector
        """

        if not quant:
            quant = list(range(1, (len(qual)+1)))

        return list(self.r.opscale(ro.IntVector(qual),
                                   ro.FloatVector(quant),
                                   level, process, na_impute,  na_assign,
                                   rescale).rx2('os'))
    

In [54]:
# First I need to encode the labels for the categorical variables in order to conduct optimal scaling.

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
targets_encoded = c1_targets.copy()

for column in targets_encoded:
    targets_encoded[column] = le.fit_transform(targets_encoded[column])

targets_encoded.head()

/Users/cguy/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


,PubFoodAs,PubHousAs,OthrFinAs,HighEdCert,CurrFTE,CurrPTE,PubFinAs,CurrenRoll,PrescripIn,Homeless,SubAbuse,Incarc,Medicaid,CnctAdult,EmplySklls,OthrHlthIn,MedicalIn,MentlHlthIn
0,4,2,2,7,2,2,2,2,4,3,2,2,4,3,2,4,4,4
1,4,2,2,5,2,2,2,3,4,2,3,3,4,3,2,4,4,4
2,4,2,2,5,2,3,2,2,4,3,2,2,4,3,2,4,4,4
3,4,2,2,5,3,2,2,2,4,3,2,2,4,3,2,5,2,4
4,3,3,2,7,2,2,3,3,4,2,2,2,4,3,2,4,4,4


In [60]:
# from pyopscale.opscale import OpScale

c1_targetsScaled = targets_encoded.copy()

os_test = OpScale()

c1_targetsScaled['PubFoodAs']

c1_targetsScaled['PubFoodAs_s'] = os_test.opscale(c1_targetsScaled['PubFoodAs'], quant=None, level=1, process=1)


In [61]:
c1_targetsScaled['PubFoodAs_s']

0       2.863064
1       2.863064
2       2.863064
3       2.863064
4      -0.972048
5      -0.972048
6       3.037596
7       2.863064
8       3.037596
9       3.037596
10      2.863064
11     -0.972048
12      2.863064
13      2.863064
14      3.037596
15      2.863064
16      3.037596
17      3.037596
18      3.037596
19      1.869771
20      3.037596
21     -0.972048
22      1.869771
23     -0.972048
24      2.863064
25     -0.972048
26     -0.972048
27      3.037596
28     -0.972048
29      3.037596
          ...   
6912    3.037596
6913    3.037596
6914    3.037596
6915    1.869771
6916    2.863064
6917    1.869771
6918    1.869771
6919    1.869771
6920   -0.972048
6921    1.869771
6922    1.685297
6923    3.037596
6924    1.869771
6925    1.869771
6926    1.869771
6927    1.869771
6928    1.869771
6929    1.869771
6930    2.863064
6931    1.869771
6932    1.869771
6933    1.869771
6934    1.869771
6935    1.869771
6936    2.863064
6937    1.869771
6938    1.869771
6939    1.8697